# 08 Word2Vec Implemented on Keras
keras is a quite high-level deep learning library. In this notebook, we are going to implement two word2vec models: CBoW and Skip-gram. The utilized corpus is IMDB movie review dataset. http://ai.stanford.edu/~amaas/data/sentiment/


## Agenda

1. How to load pre-trained word vectors
2. Reading in the IMDB Sentiment Dataset and Iterating over files in Python
3. Build Skip-gram Model
4. Build CBoW Model
5. Memory-friendly Data Generation Methods

## Part 1: Load pre-trained word vectors

- You can find the word2vec project here: https://code.google.com/archive/p/word2vec/
- Download the word embeddings from the section **Pre-trained word and phrase vectors**. It is named `GoogleNews-vectors-negative300.bin.gz (3.4G)`
- Use gensim that you can easily load these wordvectors and utilize their functions

In [1]:
from gensim.models import KeyedVectors
# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

dog = model['dog']
print(dog.shape)
print(dog[:10])

# Some predefined functions that show content related information for given words
print(model.most_similar(positive=['woman', 'king'], negative=['man']))

print(model.doesnt_match("breakfast cereal dinner lunch".split()))

print(model.similarity('woman', 'man'))

(300,)
[ 0.05126953 -0.02233887 -0.17285156  0.16113281 -0.08447266  0.05737305
  0.05859375 -0.08251953 -0.01538086 -0.06347656]
[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431607246399), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454)]
cereal
0.76640123


In [2]:
# clear the memory
del model

## Part 2: Read in the IMDB Sentiment Dataset

- You can access the imdb data folder in BT5153_data folder.
- Each movie review is a text file and they are under two different folders: pos and neg.
- We need to iterate over these files and load them one by one.

#### The data could be found in the LumiNUS->Files->Data for Week7 Notebook

In [17]:
import numpy as np
import pandas as pd
import os

In [18]:
def load_imdb_dataset(imdb_path):
    # imdb_path is the base path 
    train_texts = []
    train_labels = []
    # contain two sub-folders named pos and neg
    for cat in ['pos', 'neg']:
        dset_path = os.path.join(imdb_path, cat)
        # loop in each folder and get the file name for each txt.
        for fname in sorted(os.listdir(dset_path)):
            if fname.endswith('.txt'):
                with open(os.path.join(dset_path, fname), encoding='utf-8') as f:
                    train_texts.append(f.read()) # load the data into memory
                label = 0 if cat == 'neg' else 1
                train_labels.append(label)
    imdbdf = pd.DataFrame(
             {'text': train_texts,
              'label': train_labels}
             )
    # shuffle the whole dataset
    imdbdf = imdbdf.sample(frac=1).reset_index(drop=True)
    # Return the dataset in dataframe format
    return imdbdf

In [19]:
df_corpus = load_imdb_dataset('imdb')
print ('Train samples shape :', df_corpus.shape[0])

Train samples shape : 25000


In [20]:
# 1 denotes positive and 0 is negative
print(df_corpus.head())

                                                text  label
0  The Sarah Silverman program is very similar to...      1
1  Like many of you I am a great fan of the real ...      1
2  I have just started watching the TV series "Wh...      1
3  Not totally off the wall in a good way, but ju...      0
4  Anyone who has studied any physics or cognitiv...      0


#### Raw Text Cleaning

In [21]:
from bs4 import BeautifulSoup 
import re

def clean_txt(raw_txt):
    # Function to clean raw text
    # 1. Remove HTML
    raw_txt = BeautifulSoup(raw_txt, "html.parser").get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", raw_txt) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                                             
    # 
    #
    # 4. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( words )) 

In [22]:
df_corpus['text'] = df_corpus.text.apply(clean_txt)
corpus = df_corpus.text.tolist()

In [23]:
# check the corpus type, which is a list of string
print(type(corpus))
print(type(corpus[1]))

<class 'list'>
<class 'str'>


#### Text tokenization from Keras

This class allows to vectorize a text corpus, by turning each text into either a sequence of integers (each integer being the index of a token in a dictionary) or into a vector where the coefficient for each token could be binary, based on word count, based on tf-idf...

In [24]:
from keras.preprocessing.text import Tokenizer

In [25]:
tokenizer = Tokenizer()
# learn the vocab
tokenizer.fit_on_texts(corpus)

In [26]:
print(type(corpus[1]))
print(corpus[1])

<class 'str'>
like many of you i am a great fan of the real thing the s noir films but red rock west was a real treat for all of us longing for the past the term neo noir has been so often used inappropriately in the last ten years that it has lost its meaning and its impact john dahl s film on the other hand truly deserved to be described as such the casting is perfect all around and would have felt right at home with tay garnett or jacques tourneur the plot is so tight that you are hooked within the first fifteen minutes james m cain would have appreciated it many contemporary films leave me wondering why they don t make them like they used to and i m not even that old movies such as red rock west give us hope for the future while paying tribute to the past


This `fit_on_texts` function is trying to build the vocab

In [27]:
# from string to a sequence of intergers
# each word will be convereted to its vocab index
seq_corpus = tokenizer.texts_to_sequences(corpus)
print(seq_corpus[1])

[38, 108, 4, 21, 9, 238, 3, 84, 330, 4, 1, 145, 152, 1, 12, 1318, 106, 18, 735, 702, 1187, 13, 3, 145, 1662, 15, 30, 4, 177, 6518, 15, 1, 491, 1, 2842, 3872, 1318, 46, 76, 36, 395, 338, 14041, 8, 1, 231, 729, 154, 11, 7, 46, 405, 91, 1191, 2, 91, 1455, 296, 9121, 12, 19, 22, 1, 80, 495, 364, 1794, 5, 28, 2177, 14, 139, 1, 953, 6, 397, 30, 185, 2, 60, 27, 415, 204, 31, 339, 16, 32281, 37400, 41, 6601, 13154, 1, 111, 6, 36, 2656, 11, 21, 25, 3264, 728, 1, 83, 3484, 228, 564, 140, 6052, 60, 27, 2489, 7, 108, 2351, 106, 549, 70, 1512, 134, 32, 88, 20, 94, 95, 38, 32, 338, 5, 2, 9, 140, 23, 58, 11, 153, 98, 139, 14, 735, 702, 1187, 199, 177, 438, 15, 1, 682, 135, 2633, 3286, 5, 1, 491]


- In the following, we are going to use a toy corpus instead of the IMDB corpus for a quick demo.

In [28]:
# let us check the texts_to_sequences function
toy_corpus = ['king is a strong man', 
              'queen is a wise woman', 
              'boy is a young man',
              'girl is a young woman',
              'prince is a young king',
              'princess is a young queen',
               'man is strong', 
               'woman is pretty',
               'prince is a boy will be king',
               'princess is a girl will be queen']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(toy_corpus)
toy_seq_corpus = tokenizer.texts_to_sequences(toy_corpus)
print(toy_seq_corpus[0])
print(tokenizer.word_index['king'])
print(tokenizer.word_index['is'])
print(tokenizer.word_index['a'])
print(tokenizer.word_index['strong'])

[4, 1, 2, 8, 5]
4
1
2
8


In [29]:
print(tokenizer.index_word[1])

is


In [30]:
print(tokenizer.index_word[0])

KeyError: 0

- The KeyError means that the Tokenizer reserves 0 as an OOV words.
- In practive, the first ebmedding in word embedding martix is for unkown words or chars.

## Part 3: Build Skip-gram Model

- Here, we only use toy corpus for demo purpose.
- Target: predict the nearby words based on the center word.
<img src="imgs\word2vec-skip-gram.png" alt="cbow"
	title="cbow pic" width="250" height="150" />

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape
from keras.utils import to_categorical
from keras.preprocessing import sequence
import keras.backend as K

- **For skip-gram,  training data generation**:

the input x is the center word index, the output x is one hot vector of the neary word index.
For example, the toy corpus only contain two sentences.
```
I like apple 
I like reading books
```
1. The first step: build a vocab. which can be regarded as a mapping from words to interget index.

Here, OOV-> 0, I -> 1, like -> 2, apple -> 3, reading -> 4, books -> 5.

2. Then, we scan the corpus and creat the pair of center word and nearby word. Here, we set the window size is `one`.
We have the following pair of input x and target y.

<pre>
words pair              numerical input       numerical output

(I, like)                       1               [0,0,1,0,0,0]

(like, I)                       2               [0,1,0,0,0,0]

(like, apple)                   2               [0,0,0,1,0,0]

(apple, like)                   4               [0,0,1,0,0,0]

(I, like)

(like, I)

(like, reading)              

(reading, like)

(reading, books)

(books, reading)                 5              [0,0,0,0,1,0]
</pre>

In [32]:
def generate_data(corpus, window_size, V):
    """
    corpus is the collection of lists of words index
    window_size is the context size that defines 'nearby' words
    V is the vocab Size
    """
    labels = []
    in_words   = [] 
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            s = index - window_size
            e = index + window_size + 1
            for i in range(s, e):
                if 0<= i < L and i != index:
                    in_words.append([word])
                    labels.append(to_categorical(words[i], V))
    return (in_words, labels)   

In [33]:
# plus one is for OOV words
V = len(tokenizer.word_index) + 1
dim = 5
window_size = 4
ith = 0
input_x, target_y =  generate_data(toy_seq_corpus, window_size, V)
input_x           = np.array(input_x,dtype=np.int32)
target_y          = np.array(target_y,dtype=np.int32)

In [34]:
print('check the first pair of input and output')
print(input_x[0])
print(target_y[0])
print('check the third pair of input and output')
print(input_x[2])
print(target_y[2])

check the first pair of input and output
[4]
[0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
check the third pair of input and output
[4]
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]


In [35]:
print(toy_seq_corpus[0])

[4, 1, 2, 8, 5]


- **Model Config**

It consists of two layers:

1. The first layer is embeddings layer, which perform the lookup operation. Given the word index as the input, the layer output will return the corresponding vector

2. The second layer is softmax layer.

- **Embeddings Layer**:

Turns positive integers (indexes) into dense vectors of fixed size. eg. [[4], [20]] -> [[0.25, 0.1], [0.6, -0.2]]

This layer can only be used as the first layer in a model.

1. input_dim: int > 0. Size of the vocabulary, i.e. maximum integer index + 1.
2. output_dim: int >= 0. Dimension of the dense embedding.
3. embeddings_initializer: Initializer for the embeddings matrix (see initializers).
4. input_length: Length of input sequences, when it is constant. This argument is required if you are going to connect  Flatten then Dense layers upstream (without it, the shape of the dense outputs cannot be computed).

### Model Building

In [36]:
skipgram = Sequential()
skipgram.add(Embedding(input_dim=V, output_dim=dim, init='glorot_uniform', input_length=1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, output_dim=V, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Embedding` call to the Keras 2 API: `Embedding(input_dim=17, output_dim=5, input_length=1, embeddings_initializer="glorot_uniform")`
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=5, activation="softmax", units=17)`
  after removing the cwd from sys.path.


In [37]:
print(skipgram.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 5)              85        
_________________________________________________________________
reshape_1 (Reshape)          (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                102       
Total params: 187
Trainable params: 187
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
skipgram.compile(loss='categorical_crossentropy', optimizer="adadelta")

In [39]:
skipgram.fit(input_x, target_y, batch_size=8, epochs=30)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
204/204 [==============================] - 0s 937us/step - loss: 2.8419
Epoch 2/30
204/204 [==============================] - 0s 78us/step - loss: 2.8249
Epoch 3/30
204/204 [==============================] - 0s 98us/step - loss: 2.8091
Epoch 4/30
204/204 [==============================] - 0s 147us/step - loss: 2.7946
Epoch 5/30
204/204 [==============================] - 0s 220us/step - loss: 2.7807
Epoch 6/30
204/204 [==============================] - 0s 255us/step - loss: 2.7674
Epoch 7/30
204/204 [==============================] - 0s 348us/step - loss: 2.7548 0s - loss: 2.753
Epoch 8/30
204/204 [==============================] - 0s 211us/step - loss: 2.7432
Epoch 9/30
204/204 [==============================] - 0s 367us/step - loss: 2.7320 0s - loss: 2.740
Epoch 10/30
204/204 [==============================] - 0s 151us/step - loss: 2.7211
Epoch 11/30
204/204 [==============================] - 0s 196us/step - loss: 2.7107
Epoch

- **How to save the learned word vectors**

In [40]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(V-1, dim))

5

In [41]:
vectors = skipgram.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

- **the saved format for word vectors**
<img src="imgs\saved_format.jpg" alt="cbow"
	title="saved format" width="550" height="450" />

- **we can use gensim**

Gensim is a production-ready open-source library for NLP problems.

https://radimrehurek.com/gensim/index.html


In [42]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [43]:
w2v.most_similar(positive=['man'])

E:\Anaconda\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('will', 0.5640014410018921),
 ('a', 0.5574741363525391),
 ('young', 0.5540315508842468),
 ('princess', 0.23572158813476562),
 ('wise', 0.15920957922935486),
 ('king', 0.07441140711307526),
 ('pretty', 0.05414140224456787),
 ('boy', 0.04108387231826782),
 ('strong', 0.031051307916641235),
 ('be', -0.04768812656402588)]

## Part 4: Build CBoW Model

- CBoW's target is the prediction of center word.
<img src="imgs\word2vec-cbow.png" alt="cbow"
	title="cbow pic" width="250" height="150" />

- **For cbow,  training data generation**:

the input x is the list of  context word index, the output x is one hot vector of the center word.
For example, the toy corpus only contain two sentences.
```
I like apple 
I like reading books
```
1. The first step: build a vocab. which can be regarded as a mapping from words to interget index. 
Here OOV->0, I -> 1, like -> 2, apple -> 3, reading -> 4 books -> 5.

2. Then, we scan the corpus and creat the pair of list of nearby word and center word. Here, we set the window size is `one`.
We have the following pair of input x and target y.

<pre>
words pair                     numerical input       numerical output

([like], I)                        [2]                 [0,1,0,0,0,0]

([I, apple], like)                 [1,3]               [0,0,1,0,0,0]

([like], apple)                    [2]                 [0,0,0,1,0,0]

([like], I)                        [2]                 [0,1,0,0,0,0]
 
([I, reading], like)               [1,4]               [0,1,0,0,0,0]

([like, books], reading)           [2,5]               [0,0,0,0,1,0]

([reading], books)                 [4]                 [0,0,0,0,0,1]
</pre>

3. At last, sometimes, we can not get the input context with enough length. For example, the first pair's numerical input only has one word index insetad of two. What we can do here is padding the short input so that all input data have the same length. 

- **Prepare the training and labels**

In [44]:
from keras.preprocessing import sequence
import keras.backend as K
def generate_data(corpus, window_size, V):
    """
    corpus is the list of sequence of words index
    window_size is used to define  
    V is the vocab Size
    """
    context_words   = []
    center_words    = []
    maxlen = window_size*2
    for words in corpus:
        L = len(words)
        for index, word in enumerate(words):
            contexts = []
            labels   = []            
            s = index - window_size
            e = index + window_size + 1
            contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
            labels.append(word)           
            x = sequence.pad_sequences(contexts, maxlen=maxlen)
            y = to_categorical(labels, V)
            context_words.append(x)
            center_words.append(y)
    return context_words, center_words

In [45]:
ith = 0
input_x, target_y = generate_data(toy_seq_corpus, window_size, V)

In [46]:
input_x = np.array(input_x)
print(input_x.shape)
input_x = np.squeeze(input_x)  # sequeeze the second dimesion as on
print(input_x.shape)
target_y = np.array(target_y)
target_y = np.squeeze(target_y)

(50, 1, 8)
(50, 8)


In [47]:
print(input_x[0])
print(target_y[0])

[0 0 0 0 1 2 8 5]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [48]:
print(toy_seq_corpus[0])

[4, 1, 2, 8, 5]


In [49]:
cbow = Sequential()

- **Lambda Layer**:

Wraps arbitrary expression as a Layer object.
    1. function: The function to be evaluated. Takes input tensor as first argument. usually based on backend
    2. output_shape: Expected output shape from function. 

In [50]:
from keras.layers import Lambda
cbow.add(Embedding(input_dim=V, output_dim=dim, input_length=window_size*2))
# sum all embeddings 
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(dim,)))
## add softmax layer
cbow.add(Dense(V, activation='softmax'))

In [51]:
print(cbow.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 8, 5)              85        
_________________________________________________________________
lambda_1 (Lambda)            (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 17)                102       
Total params: 187
Trainable params: 187
Non-trainable params: 0
_________________________________________________________________
None


In [52]:
cbow.compile(loss='categorical_crossentropy', optimizer='adam')
# Train the model, iterating on the data in batches of 512 samples
cbow.fit(input_x, target_y, batch_size=8, epochs=10)

Epoch 1/10
50/50 [==============================] - 0s 4ms/step - loss: 2.8282
Epoch 2/10
50/50 [==============================] - 0s 120us/step - loss: 2.8245
Epoch 3/10
50/50 [==============================] - 0s 100us/step - loss: 2.8203
Epoch 4/10
50/50 [==============================] - 0s 140us/step - loss: 2.8168
Epoch 5/10
50/50 [==============================] - 0s 120us/step - loss: 2.8131
Epoch 6/10
50/50 [==============================] - 0s 120us/step - loss: 2.8094
Epoch 7/10
50/50 [==============================] - 0s 160us/step - loss: 2.8058
Epoch 8/10
50/50 [==============================] - 0s 140us/step - loss: 2.8023
Epoch 9/10
50/50 [==============================] - 0s 100us/step - loss: 2.7985
Epoch 10/10
50/50 [==============================] - 0s 120us/step - loss: 2.7952


## Part 5: Memory-friendly Data Generation

- Here, we modify the data generation function of skip-gram
- `yield`: it will return generators. And generators do not store all the values in memory. It will return value during each iteration.

sample code
```
generator = (x * x for x in range(3))
for i in generator:
    print(i)
```

https://stackoverflow.com/questions/231767/what-does-the-yield-keyword-do


In [53]:
def generate_data_live(corpus, window_size, V):
    """
    corpus is the list of sequence of words index
    window_size is used to define  
    V is the vocab Size
    """
    maxlen = window_size*2
    for words in corpus:
        labels   = []
        in_words = [] 
        L = len(words)
        for index, word in enumerate(words):
            s = index - window_size
            e = index + window_size + 1
            for i in range(s, e):
                if 0<= i < L and i != index:
                    in_words.append([word])
                    labels.append(words[i])
        x = np.array(in_words,dtype=np.int32)
        y = to_categorical(labels, V)
        yield (x, y) 


In [54]:
## here you should define skipgram from scratch

for ite in range(50):
    loss = 0.
    for x, y in generate_data_live(toy_seq_corpus, window_size, V):
        #updated parameters based on data samples provided without regard to any fixed batch size
        loss += skipgram.train_on_batch(x, y)
    print(ite, loss)

0 25.845551013946533
1 25.818074464797974
2 25.79086971282959
3 25.763766288757324
4 25.736700773239136
5 25.709627151489258
6 25.68252992630005
7 25.655402660369873
8 25.628257274627686
9 25.601116180419922
10 25.57400608062744
11 25.546953439712524
12 25.519986391067505
13 25.493131399154663
14 25.466410636901855
15 25.439845085144043
16 25.413453340530396
17 25.38724970817566
18 25.361250638961792
19 25.335467100143433
20 25.309908628463745
21 25.284584045410156
22 25.25950336456299
23 25.234670877456665
24 25.210091829299927
25 25.185771465301514
26 25.161712884902954
27 25.137916803359985
28 25.11438775062561
29 25.091126441955566
30 25.06813359260559
31 25.04540777206421
32 25.022951364517212
33 25.000761032104492
34 24.978839635849
35 24.95718479156494
36 24.93579363822937
37 24.91466760635376
38 24.893802404403687
39 24.873197078704834
40 24.85284996032715
41 24.83275818824768
42 24.812920331954956
43 24.793333292007446
44 24.77399444580078
45 24.754902124404907
46 24.736051797